# Section 4

Today:
<ul>
<li>Convolutions</li>
<li>Pooling</li>
<li>TensorBoard</li>
</ul>

<em>Some code from tensorflow.org documentation.</em>

## Extra Resources
<ul>
<li><a href='https://www.tensorflow.org/api_docs/python/tf/name_scope'>TF Name Scoping</a></li>
<li><a href='http://cs231n.github.io/convolutional-networks/'>CS 231n Notes on CNNs</a></li>
</ul>

## Convolutions and Pooling
<a href='https://www.tensorflow.org/tutorials/layers'>A Guide to TF Layers: Building a Convolutional Neural Network</a><br/>
<a href='https://www.tensorflow.org/versions/r1.3/api_docs/python/tf/layers/conv2d'>tf.layers.conv2d</a><br/>
<a href='https://www.tensorflow.org/api_docs/python/tf/layers/max_pooling2d'>tf.layers.max_pooling2d</a><br/>

In [1]:
# Setup and load data
import tensorflow as tf
import numpy as np

sess = tf.Session()

def load(filename, W=64, H=64):
    data = np.fromfile(filename, dtype=np.uint8).reshape((-1, W*H*3+1))
    images, labels = data[:, :-1].reshape((-1,H,W,3)), data[:, -1]
    return images, labels

train_image_data, train_label_data = load('tux_train.dat')
val_image_data, val_label_data = load('tux_val.dat')

In [ ]:
# Let's build our first CNN





## TensorBoard
<a href='https://www.tensorflow.org/get_started/summaries_and_tensorboard'>TensorBoard Introduction</a><br/>
<a href='https://www.tensorflow.org/api_guides/python/summary'>Summary variables</a><br/>
<a href='https://www.tensorflow.org/api_docs/python/tf/summary/merge_all'>merge_all</a><br/>
<a href='https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter'>FileWriter</a><br/>
<a href='https://research.googleblog.com/2017/09/build-your-own-machine-learning.html'>New TensorBoard API</a><br/><br/>
Let's use the graph we used when introducing convolutions but now add summary operations so we can see the graph and monitor training.


In [ ]:
# First let's set up TensorBoard


# Merge them together


In [4]:
# Add some summary operations

tf.reset_default_graph()

sess = tf.Session()

# Let's build our first CNN
I = tf.placeholder(tf.float32, (None, 64, 64, 3))
white_inputs = (I - 100.) / 72.
labels = tf.placeholder(tf.int64, (None))

# Convolutional Layer #1
conv1 = tf.contrib.layers.conv2d(
  inputs=white_inputs,
  num_outputs=20,
  kernel_size=[5, 5],
  stride=2,
  padding="same",
  scope='conv1')
print(conv1)

conv2 = tf.contrib.layers.conv2d(
  inputs=white_inputs,
  num_outputs=64,
  kernel_size=[5, 5],
  stride=2,
  padding="same",
  scope='conv2')
print(conv2)

# Pooling Layer #1
pool1 = tf.contrib.layers.max_pool2d(inputs=conv2, kernel_size=[2, 2], stride=2, scope='pool1')
print(pool1)



Tensor("conv1/Relu:0", shape=(?, 32, 32, 20), dtype=float32)
Tensor("conv2/Relu:0", shape=(?, 32, 32, 64), dtype=float32)
Tensor("pool1/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)


In [ ]:
# Start TensorBoard from the command line
# tensorboard --logidr=/tmp/section4

In [ ]:
# Train your network and watch it on TensorBoard
BS = 32
train_count = 0
for epoch in range(20):
    np.random.seed(epoch)
    np.random.shuffle(train_image_data)
    np.random.seed(epoch)
    np.random.shuffle(train_label_data)
    # Validation Accuracy
    summary, acc = sess.run([merged, accuracy], feed_dict={I: val_image_data, labels: val_label_data})
    validation_writer.add_summary(summary, train_count)
    print('Validation accuracy at epoch %s: %s' % (epoch, acc))
    # Train
    # Go through the entire dataset once
    for i in range(0, train_image_data.shape[0]-BS+1, BS):
        # Train a single batch
        batch_images, batch_labels = train_image_data[i:i+BS], train_label_data[i:i+BS]
        summary, _ = sess.run([merged, train_op], feed_dict={I: batch_images, labels: batch_labels})
        train_writer.add_summary(summary, train_count)
        train_count += 1
    validation_writer.flush()
    train_writer.flush()
    